In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 1/DATASET/train.csv")
test = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 1/DATASET/test.csv")

In [3]:
import re
def postprocess_data(train_df, test_df, predict):
    
    test_df['Country.of.Origin'] = predict
    # Altitude 후처리
    test_df['Altitude'] = test_df['Altitude'].str.replace('.', '').replace(' ', '')

    def map_country_based_on_altitude(country, altitude_text):
        if isinstance(altitude_text, str):  # 문자열인지 확인
            if 'msn' in altitude_text:
                if country not in [1, 2]:
                    return 1
            elif 'psn' in altitude_text or 'ft' in altitude_text:
                return 2
        return country

    test_df['Country.of.Origin'] = test_df.apply(lambda row: map_country_based_on_altitude(row['Country.of.Origin'], row['Altitude']), axis=1)

    # Farm.Name 후처리
    farm_country_mapping = train_df[train_df['Farm.Name'].isin(['santa maria', 'several', 'various']) == False].groupby('Farm.Name')['Country.of.Origin'].first().to_dict()

    def postprocess_country(df, farm_country_mapping):
        for index, row in df.iterrows():
            farm_name = row['Farm.Name']
            if farm_name in farm_country_mapping:
                df.at[index, 'Country.of.Origin'] = farm_country_mapping[farm_name]
        return df

    test_df = postprocess_country(test_df, farm_country_mapping)

    # Lot.Number 후처리
    def map_country_of_origin_postprocessing(value):
        if re.match(r'^\d+/\d+[A-Za-z]$', str(value)) or str(value).startswith('431') or re.match(r'^\d{3}/\d{2}$', str(value)) or 'Lot' in str(value):
            return 0
        elif re.match(r'^3-\d{2}-\d{4}$', str(value)) or re.match(r'^\d{2}-\d{4}$', str(value)):
            return 1
        elif re.match(r'^11/\d+/\d+$', str(value)) or re.match(r'^11-\d+-\d+$', str(value)):
            return 2

    def check_country_of_origin(value):
        pattern = (
            r'^\d+/\d+[A-Za-z]$|^431|^(\d{3}/\d{2})|Lot'  # Condition 1
            r'|^3-\d{2}-\d{4}$|^\d{2}-\d{4}$'  # Condition 2
            r'|^11/\d+/\d+$|^11-\d+-\d+$'  # Condition 3
        )

        if re.match(pattern, str(value)):
            return True
        else:
            return False

    def postprocess_country_of_origin(df):
        for index, row in df.iterrows():
            if check_country_of_origin(row['Lot.Number']):
                df.at[index, 'Country.of.Origin'] = map_country_of_origin_postprocessing(row['Lot.Number'])
        return df

    test_df = postprocess_country_of_origin(test_df)

    return test_df['Country.of.Origin'].values


In [4]:
test["Country.of.Origin"] = np.nan

In [5]:
test["Country.of.Origin"] = postprocess_data(train, test, test["Country.of.Origin"])

In [8]:
test[test["Country.of.Origin"].isna()]

,Unnamed: 0,Species,Farm.Name,Lot.Number,Altitude,Number.of.Bags,Bag.Weight,Harvest.Year,Grading.Date,Variety,...,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Country.of.Origin
3,3,Arabica,santa bárbara,2016/0002,1020,305,59 kg,2016 / 2017,"October 26th, 2016",Catuai,...,10.0,7.50,81.92,0.11,0,0,Green,4,"October 26th, 2017",NaN
4,4,Arabica,el encinal,NaN,975,10,1 kg,2012,"September 27th, 2012",Caturra,...,10.0,7.33,79.67,0.13,5,0,NaN,6,"September 27th, 2013",NaN
7,7,Arabica,caltuchoco,NaN,1100,10,1 kg,2012,"July 26th, 2012",Typica,...,10.0,7.50,82.50,0.12,0,0,Green,5,"July 26th, 2013",NaN
8,8,Arabica,la vuelta,NaN,800,10,1 kg,2012,"September 10th, 2012",Mundo Novo,...,10.0,6.58,71.08,0.12,4,0,Green,24,"September 10th, 2013",NaN
9,9,Arabica,productor independiente guadalupe victoria,NaN,1600,250,1 kg,2012,"August 30th, 2012",Typica,...,10.0,7.17,81.50,0.12,0,0,Green,5,"August 30th, 2013",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,135,Arabica,el limon,NaN,NaN,275,1 kg,2012,"March 26th, 2012",Catuai,...,10.0,7.67,84.17,0.11,0,0,Bluish-Green,2,"March 26th, 2013",NaN
136,136,Arabica,various,NaN,1200 m,250,6,08/09 crop,"April 9th, 2010",Typica,...,10.0,6.58,76.75,0.00,0,0,NaN,0,"April 9th, 2011",NaN
139,139,Arabica,various,NaN,1300,200,1 kg,2013,"March 29th, 2013",Bourbon,...,10.0,7.75,83.25,0.12,0,0,Green,6,"March 29th, 2014",NaN
143,143,Arabica,NaN,NaN,NaN,250,70 kg,NaN,"February 1st, 2011",NaN,...,10.0,7.58,83.42,0.10,1,0,NaN,3,"February 1st, 2012",NaN


In [ ]:
test[test["Country.of.Origin"].isna()].to_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 1/DATASET/test_1.csv", index = False)

In [ ]:
test[~test["Country.of.Origin"].isna()].to_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 1/DATASET/test_2.csv", index = False)